### evaluating the output forecastings

based on: Evaluate-forecasts-paper-v5.ipynb

output of forecastings format:

    carno	startlap	startrank	endrank	diff	sign	pred_endrank	pred_diff	pred_sign	endlap	pred_endlap
      11	12	31	3.0	5.0	2.0	1	1.0	-2.0	-1	49	58
    
refer to:
    19.RankNet/stage_model_regressor.ipynb
    19.RankNet/RankForecasting-stint-paper-1kpitmodel.ipynb

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os,sys
import mxnet as mx
from mxnet import gluon
import pickle
import json
import random
import inspect
from scipy import stats
from sklearn.metrics import mean_squared_error
from gluonts.dataset.common import ListDataset
from gluonts.dataset.util import to_pandas
from pathlib import Path
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

#from indycar.model.stint_predictor_fastrun import *
#import indycar.model.stint_simulator as stint
#import indycar.model.stint_simulator_shortterm_pitmodel as stint
#import indycar.model.stint_simulator_paper as stint

import indycar.model.quicktest_simulator as stint

# import all functions 
#from indycar.model.global_variables import _hi
import indycar.model.global_variables as gvar
from indycar.model.quicktest_modules import *

INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU


In [2]:
def load_dfout(datafile, basedir = './data/'):
    datafile = basedir + datafile  
    #with open('laptime_rank_timediff_fulltest-oracle-%s.pickle'%year, 'rb') as f:
    with open(datafile, 'rb') as f:
        # The protocol version used is detected automatically, so we do not
        # have to specify it.
        dfout = pickle.load(f, encoding='latin1') 
        
        return dfout[0]
    
def load_dfout_all(datafile):
    #with open('laptime_rank_timediff_fulltest-oracle-%s.pickle'%year, 'rb') as f:
    datafile = '../result/22.PaperFinal/' + datafile  
    with open(datafile, 'rb') as f:
        # The protocol version used is detected automatically, so we do not
        # have to specify it.
        dfout = pickle.load(f, encoding='latin1') 
        
        return dfout

## 1. evalute stint restuls

In [3]:
_inlap_status = 0
_featureCnt = 9
#
# input data parameters
#
# event -> car#, maxlap
_race_info = {}
# the races have 7 years data 
races = ['Indy500', 'Texas','Iowa','Pocono']
years = ['2013','2014','2015','2016','2017','2018','2019']

events = []
for race in races:
    events.extend([f'{race}-{x}' for x in years])

events.extend(['Phoenix-2018','Gateway-2018','Gateway-2019'])
events_id={key:idx for idx, key in enumerate(events)}

# dataset shared
dataOutputRoot = "data/"
dataroot = dataOutputRoot
covergap = 1
dbid = f'IndyCar_d{len(events)}_v{_featureCnt}_p{_inlap_status}'
LAPTIME_DATASET = f'{dataOutputRoot}/laptime_rank_timediff_pit-oracle-{dbid}.pickle' 
STAGE_DATASET = f'{dataOutputRoot}/stagedata-{dbid}.pickle' 
PITCOVERED_DATASET = f'{dataOutputRoot}/pitcoveredlaps-{dbid}-g{covergap}.pickle'
PITSTOP_DATASET = f'{dataOutputRoot}/pitstop-{dbid}.csv' 
    
print('Load laptime and stage dataset:',LAPTIME_DATASET, STAGE_DATASET)
with open(LAPTIME_DATASET, 'rb') as f:
    global_carids, laptime_data = pickle.load(f, encoding='latin1') 
with open(STAGE_DATASET, 'rb') as f:
    #stagedata = pickle.load(f, encoding='latin1') 
    stagedata_o, _race_info, _events, _events_id = pickle.load(f, encoding='latin1') 
pitstop_df = pd.read_csv(PITSTOP_DATASET)

with open(PITCOVERED_DATASET, 'rb') as f:
    pitdata = pickle.load(f, encoding='latin1')

#check it
if not _events == events:
    print('Error, events mismatch at:', STAGE_DATASET)
    sys.exit(-1)

#trainrace = 'Indy500','Iowa','Pocono','Texas'
trainrace = 'Indy500'
    
#_train_events = [events_id[x] for x in [f'{trainrace}-{x}' for x in ['2013','2014','2015','2016','2017']]]
#_test_events = [f'{trainrace}-{x}' for x in ['2018','2019']]
if trainrace == 'Pocono':
    # 2014, 2019 is bad
    _train_years = ['2013','2015','2016','2017']
    _test_events = [f'{trainrace}-2018']
else:
    _train_years = ['2013','2014','2015','2016','2017']
    _test_events = [f'{trainrace}-2018',f'{trainrace}-2019']
    
_train_events = [events_id[x] for x in [f'{trainrace}-{x}' for x in _train_years]]

Load laptime and stage dataset: data//laptime_rank_timediff_pit-oracle-IndyCar_d31_v9_p0.pickle data//stagedata-IndyCar_d31_v9_p0.pickle


In [4]:
gvar.events = events
gvar.events_id  = events_id
#gvar.events_info = events_info
gvar._race_info = _race_info


In [5]:
#load data
_trim = 0
_include_final = True
_include_stintlen = True
#_include_stintlen = False
include_str = '1' if _include_final else '0'
stint_str = '1' if _include_stintlen else ''

plen = 2
_context_len = 60

version = f'IndyCar-d{trainrace}'


## 3. Split by pit-covered-laps and normal laps

define pit covered laps := two laps before and after a pit

split shortterm-results only

In [6]:
def clear_samples(full_samples, full_tss, clearidx):
    """
    clear the laps in clearidx
    """
    
    carlist = full_tss.keys()
    
    for carid, carno in enumerate(carlist):
        forecast = full_samples[carno]
        target = full_tss[carno]
        
        forecast[:, clearidx] = np.nan
        target[clearidx] = np.nan
        
        full_samples[carno] = forecast
        full_tss[carno] = target
        
    return full_samples, full_tss

In [7]:
#
# Model,SignAcc,MAE,50-Risk,90-Risk
# 
cols = ['Year','Model','ExpID', 'LapType', 'Top1Acc','SignAcc','MAE','50-Risk','90-Risk']
models = {'currank':'CurRank','rf':'RandomForest','svr':'SVM','xgb':'XGBoost'}

#plen=2
#usemeanstr='median'
usemeanstr='mean'
#stint_split_result = {}

In [8]:
retdata = []
   
for plen in [2]:
    outfile=f'{dataroot}/shortterm-dfout-mlmodels-{version}-end{include_str}-rerank-t{plen}-c{_context_len}.pickle'
    shortterm_df = load_dfout(outfile, basedir = './')

    #for year in ['Indy500-2018','Indy500-2019','Phoenix-2018']:
    #for year in [_test_event]:
    for year in _test_events:
        
        testevent = year
        gvar.maxlap = get_event_info(testevent)[2]
        
        for clf in ['currank','rf','svr','xgb']:
            print('year:',year,'clf:',clf)
            #dfout, accret = eval_sync(preddf[year][clf],errlist[year])
            dfout = shortterm_df[year][clf]
            accret = stint.get_evalret_shortterm(dfout)[0]

            fsamples, ftss = df2samples(dfout)
            _, prisk_vals = prisk_direct_bysamples(fsamples, ftss)

            retdata.append([year,models[clf],'MLMODELS','all', accret[0], accret[4], accret[1], prisk_vals[1], prisk_vals[2]])
            
for laptype in ['normal','pit']:
    
    plen = 2
    
    outfile=f'{dataroot}/shortterm-dfout-mlmodels-{version}-end{include_str}-rerank-t{plen}-c{_context_len}.pickle'
    shortterm_df = load_dfout(outfile, basedir = './')


    #for year in ['2018','2019']:
    #for year in [_test_event]:
    for year in _test_events:
        
        
        testevent = year
        gvar.maxlap = get_event_info(testevent)[2]
        
        # select the set
        pitcoveredlaps = pitdata[year][1]
        normallaps = set([x for x in range(1,201)]) - pitcoveredlaps
        
        if laptype == 'normal':
            sellaps = normallaps
            clearlaps = pitcoveredlaps
        else:
            sellaps = pitcoveredlaps
            clearlaps = normallaps
        
        
        # pitcoveredlaps start idx = 1
        startlaps = [x-plen-1 for x in sellaps]
        #sellapidx = np.array([x-1 for x in sellaps])
        clearidx = np.array([x-1 for x in clearlaps])
        
        for clf in ['currank','rf','svr','xgb']:
            print('year:',year,'clf:',clf)
            
            dfout = shortterm_df[year][clf]
            fsamples, ftss = df2samples(dfout)
            
            fsamples, ftss = clear_samples(fsamples, ftss,clearidx)
            _, prisk_vals = prisk_direct_bysamples(fsamples, ftss)
            
            # split dfout by startlap
            # curlap = int(dfrec.startlap.values[0] + prediction_len)
            dfout = dfout[dfout['startlap'].isin(startlaps)]
            accret = stint.get_evalret_shortterm(dfout)[0]

            retdata.append([year,models[clf],'MLMODELS',laptype, accret[0], accret[4], accret[1], prisk_vals[1], prisk_vals[2]])
            
    #print(stint_split_result[laptype])
retdf = pd.DataFrame(data=retdata, columns=cols) 
retdf.to_csv(f'{dataroot}/evaluation_result_shortterm_{trainrace}_plen2-split-g{covergap}_v6-c{_context_len}.csv')

FileNotFoundError: [Errno 2] No such file or directory: './data//shortterm-dfout-mlmodels-IndyCar-dIndy500-end1-rerank-t2-c60.pickle'

In [ ]:
retdf